In [1]:
import pandas as pd
import pyreadr
import numpy as np
import datetime

In [2]:
df = pyreadr.read_r('./data_eval/data-shared/20210317_processed.RDS')[None]
df

,timestamp,reporting_id,opposing_id,distance,condition_id,group_id
0,2021-03-17 12:21:35,210516_2021-03-17_12:19:49_12:31:52,210562_2021-03-17_12:21:33_12:27:25,493.0,1.0,NaN
1,2021-03-17 12:21:35,210562_2021-03-17_12:21:33_12:27:25,210516_2021-03-17_12:19:49_12:31:52,493.0,1.0,NaN
2,2021-03-17 12:21:36,210516_2021-03-17_12:19:49_12:31:52,210562_2021-03-17_12:21:33_12:27:25,430.0,1.0,NaN
3,2021-03-17 12:21:36,210562_2021-03-17_12:21:33_12:27:25,210516_2021-03-17_12:19:49_12:31:52,430.0,1.0,NaN
4,2021-03-17 12:21:37,210516_2021-03-17_12:19:49_12:31:52,210562_2021-03-17_12:21:33_12:27:25,387.0,1.0,NaN
...,...,...,...,...,...,...
366277,2021-03-17 16:45:00,210788_2021-03-17_16:37:54_16:45:00,210668_2021-03-17_16:42:30_16:45:00,366.0,1.0,NaN
366278,2021-03-17 16:45:00,210988_2021-03-17_16:30:17_16:45:00,208402_2021-03-17_16:37:09_16:45:00,464.0,1.0,NaN
366279,2021-03-17 16:45:00,210988_2021-03-17_16:30:17_16:45:00,208711_2021-03-17_16:41:18_16:45:00,381.0,1.0,NaN
366280,2021-03-17 16:45:00,210988_2021-03-17_16:30:17_16:45:00,210436_2021-03-17_16:24:53_16:45:00,339.0,1.0,NaN


In [4]:
df = pyreadr.read_r('./data_eval/data-shared/20210317_processed.RDS')[None]

# Get unique nodes
unique_nodes = np.unique(df[['reporting_id', 'opposing_id']].values.flatten())
inds = np.arange(0, len(unique_nodes), 1)

# Get index like node names
node_int_dict = dict(zip(unique_nodes, inds))

# Map nodes to index
df['i'] = df.reporting_id.map(node_int_dict)
df['j'] = df.opposing_id.map(node_int_dict)

# Extract start and end times from reporting_id and opposing_id
df[['i_date', 'i_start', 'i_end']] = df['reporting_id'].str.split('_', expand=True).iloc[:, [1, 2, 3]]
df[['j_date', 'j_start', 'j_end']] = df['opposing_id'].str.split('_', expand=True).iloc[:, [1, 2, 3]]

# Combine date with start and end times
df['i_start'] = pd.to_datetime(df['i_date'] + ' ' + df['i_start'])
df['i_end'] = pd.to_datetime(df['i_date'] + ' ' + df['i_end'])
df['j_start'] = pd.to_datetime(df['j_date'] + ' ' + df['j_start'])
df['j_end'] = pd.to_datetime(df['j_date'] + ' ' + df['j_end'])

# Drop intermediate date columns
df.drop(['i_date', 'j_date'], axis=1, inplace=True)

# Find the minimum timestamp
min_timestamp = df[['i_start', 'i_end', 'j_start', 'j_end', 'timestamp']].min().min()

# Normalize temporal columns to seconds since the smallest occurred time
df['i_start_seconds'] = (df['i_start'] - min_timestamp).dt.total_seconds().astype('int')
df['i_end_seconds'] = (df['i_end'] - min_timestamp).dt.total_seconds().astype('int')
df['j_start_seconds'] = (df['j_start'] - min_timestamp).dt.total_seconds().astype('int')
df['j_end_seconds'] = (df['j_end'] - min_timestamp).dt.total_seconds().astype('int')
df['t'] = (df['timestamp'] - min_timestamp).dt.total_seconds().astype('int')

# Display the resulting DataFrame
df

,timestamp,reporting_id,opposing_id,distance,condition_id,group_id,i,j,i_start,i_end,j_start,j_end,i_start_seconds,i_end_seconds,j_start_seconds,j_end_seconds,t
0,2021-03-17 12:21:35,210516_2021-03-17_12:19:49_12:31:52,210562_2021-03-17_12:21:33_12:27:25,493.0,1.0,NaN,265,290,2021-03-17 12:19:49,2021-03-17 12:31:52,2021-03-17 12:21:33,2021-03-17 12:27:25,0,723,104,456,106
1,2021-03-17 12:21:35,210562_2021-03-17_12:21:33_12:27:25,210516_2021-03-17_12:19:49_12:31:52,493.0,1.0,NaN,290,265,2021-03-17 12:21:33,2021-03-17 12:27:25,2021-03-17 12:19:49,2021-03-17 12:31:52,104,456,0,723,106
2,2021-03-17 12:21:36,210516_2021-03-17_12:19:49_12:31:52,210562_2021-03-17_12:21:33_12:27:25,430.0,1.0,NaN,265,290,2021-03-17 12:19:49,2021-03-17 12:31:52,2021-03-17 12:21:33,2021-03-17 12:27:25,0,723,104,456,107
3,2021-03-17 12:21:36,210562_2021-03-17_12:21:33_12:27:25,210516_2021-03-17_12:19:49_12:31:52,430.0,1.0,NaN,290,265,2021-03-17 12:21:33,2021-03-17 12:27:25,2021-03-17 12:19:49,2021-03-17 12:31:52,104,456,0,723,107
4,2021-03-17 12:21:37,210516_2021-03-17_12:19:49_12:31:52,210562_2021-03-17_12:21:33_12:27:25,387.0,1.0,NaN,265,290,2021-03-17 12:19:49,2021-03-17 12:31:52,2021-03-17 12:21:33,2021-03-17 12:27:25,0,723,104,456,108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366277,2021-03-17 16:45:00,210788_2021-03-17_16:37:54_16:45:00,210668_2021-03-17_16:42:30_16:45:00,366.0,1.0,NaN,475,376,2021-03-17 16:37:54,2021-03-17 16:45:00,2021-03-17 16:42:30,2021-03-17 16:45:00,15485,15911,15761,15911,15911
366278,2021-03-17 16:45:00,210988_2021-03-17_16:30:17_16:45:00,208402_2021-03-17_16:37:09_16:45:00,464.0,1.0,NaN,484,21,2021-03-17 16:30:17,2021-03-17 16:45:00,2021-03-17 16:37:09,2021-03-17 16:45:00,15028,15911,15440,15911,15911
366279,2021-03-17 16:45:00,210988_2021-03-17_16:30:17_16:45:00,208711_2021-03-17_16:41:18_16:45:00,381.0,1.0,NaN,484,84,2021-03-17 16:30:17,2021-03-17 16:45:00,2021-03-17 16:41:18,2021-03-17 16:45:00,15028,15911,15689,15911,15911
366280,2021-03-17 16:45:00,210988_2021-03-17_16:30:17_16:45:00,210436_2021-03-17_16:24:53_16:45:00,339.0,1.0,NaN,484,163,2021-03-17 16:30:17,2021-03-17 16:45:00,2021-03-17 16:24:53,2021-03-17 16:45:00,15028,15911,14704,15911,15911


In [43]:
# Convert time strings to seconds
def time_to_seconds(time_str):
    hours, minutes, seconds = map(int, time_str.split(':'))
    return hours * 3600 + minutes * 60 + seconds

In [93]:
df = pyreadr.read_r('./data_eval/data-shared/20210317_processed.RDS')[None]

# Get unique nodes
unique_nodes = np.unique(df[['reporting_id', 'opposing_id']].values.flatten())
inds = np.arange(0, len(unique_nodes), 1)

# Check if all strings have the same length
same_length = np.all(np.vectorize(len)(unique_nodes) == len(unique_nodes[0]))

if same_length:
    # Assuming you want to extract a slice of length 2 from each string
    start = np.vectorize(lambda s: s[18:26])(unique_nodes)
    end =  np.vectorize(lambda s: s[27:])(unique_nodes)
else:
    print("Strings have different lengths.")

# Apply the conversion function to each element in the array
start = np.vectorize(time_to_seconds)(start)
t0 = start.min()
start = start - t0
end = np.vectorize(time_to_seconds)(end)
end = end - t0

# Create dictionaries
node_start_dict = dict(zip(inds, start))
node_end_dict = dict(zip(inds, end))

# Get index like node names
node_int_dict = dict(zip(unique_nodes, inds))

# Map nodes to index
df['i'] = df.reporting_id.map(node_int_dict)
df['j'] = df.opposing_id.map(node_int_dict)

# Get node start and end time of node trajectory
df['i_start'] = df.i.map(node_start_dict)
df['i_end'] = df.i.map(node_end_dict)
df['j_start'] = df.j.map(node_start_dict)
df['j_end'] = df.j.map(node_end_dict)

# Drop contacts smaller than specified distance
df = df[df.distance <= 200.0]

# Convert to sociopatterns time format
df['day'] = df.timestamp.dt.date
df['date'] = df.timestamp

df['t'] = (df.timestamp - df.timestamp.iloc[0]).dt.total_seconds()
df


,timestamp,reporting_id,opposing_id,distance,condition_id,group_id,i,j,i_start,i_end,j_start,j_end,day,date,t
40,2021-03-17 12:21:47,210451_2021-03-17_12:21:15_12:29:19,210562_2021-03-17_12:21:33_12:27:25,141.0,1.0,NaN,177,290,86,570,104,456,2021-03-17,2021-03-17 12:21:47,0.0
43,2021-03-17 12:21:47,210562_2021-03-17_12:21:33_12:27:25,210451_2021-03-17_12:21:15_12:29:19,141.0,1.0,NaN,290,177,104,456,86,570,2021-03-17,2021-03-17 12:21:47,0.0
44,2021-03-17 12:21:48,210451_2021-03-17_12:21:15_12:29:19,210562_2021-03-17_12:21:33_12:27:25,141.0,1.0,NaN,177,290,86,570,104,456,2021-03-17,2021-03-17 12:21:48,1.0
47,2021-03-17 12:21:48,210562_2021-03-17_12:21:33_12:27:25,210451_2021-03-17_12:21:15_12:29:19,141.0,1.0,NaN,290,177,104,456,86,570,2021-03-17,2021-03-17 12:21:48,1.0
48,2021-03-17 12:21:49,210451_2021-03-17_12:21:15_12:29:19,210562_2021-03-17_12:21:33_12:27:25,85.0,1.0,NaN,177,290,86,570,104,456,2021-03-17,2021-03-17 12:21:49,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366252,2021-03-17 16:45:00,210417_2021-03-17_16:33:22_16:45:00,210788_2021-03-17_16:37:54_16:45:00,199.0,1.0,NaN,143,475,15213,15911,15485,15911,2021-03-17,2021-03-17 16:45:00,15793.0
366253,2021-03-17 16:45:00,210427_2021-03-17_16:22:22_16:45:00,210762_2021-03-17_16:28:05_16:45:00,95.0,1.0,1.30,151,443,14553,15911,14896,15911,2021-03-17,2021-03-17 16:45:00,15793.0
366269,2021-03-17 16:45:00,210762_2021-03-17_16:28:05_16:45:00,210427_2021-03-17_16:22:22_16:45:00,95.0,1.0,1.30,443,151,14896,15911,14553,15911,2021-03-17,2021-03-17 16:45:00,15793.0
366272,2021-03-17 16:45:00,210788_2021-03-17_16:37:54_16:45:00,208402_2021-03-17_16:37:09_16:45:00,190.0,1.0,NaN,475,21,15485,15911,15440,15911,2021-03-17,2021-03-17 16:45:00,15793.0


In [38]:
# Load data
df = pyreadr.read_r('./data_eval/data-shared/20210317_processed.RDS')[None]

# Get unique nodes
unique_nodes = np.unique(df[['reporting_id', 'opposing_id']].values.flatten())

# Get index like node names
node_int_dict = dict(zip(unique_nodes, inds))

# get node start and end time of node trajectory
# TODO

# Map nodes to index
df.reporting_id = df.reporting_id.map(node_int_dict)
df.opposing_id = df.opposing_id.map(node_int_dict)

# Drop contacts smaller than specified distance
df = df[df.distance <= 200.0]

# Convert to sociopatterns time format
df['day'] = df.timestamp.dt.date
df['date'] = df.timestamp
df.timestamp = (df.timestamp - df.timestamp.min()).dt.seconds
df.timestamp.max()

15793

In [40]:

df_ps = pd.read_parquet('./data_eval_split/gallery/f20_2009-05-22.parquet')
df_ps

,t,i,j,date,day
0,0,70,168,2009-05-22 10:04:19,2009-05-22
1,0,72,172,2009-05-22 10:04:19,2009-05-22
2,1,70,168,2009-05-22 10:04:39,2009-05-22
3,1,72,172,2009-05-22 10:04:39,2009-05-22
4,2,70,168,2009-05-22 10:04:59,2009-05-22
...,...,...,...,...,...
5524,1442,148,132,2009-05-22 18:04:59,2009-05-22
5525,1442,148,153,2009-05-22 18:04:59,2009-05-22
5526,1442,132,153,2009-05-22 18:04:59,2009-05-22
5527,1443,148,132,2009-05-22 18:05:19,2009-05-22


In [9]:
# complete Berlin data
df_berlin = pd.read_csv('./VF_data/pandemos_tra_tapas_modell.csv')

In [11]:
# export important Berlin data to parquet
pd.read_parquet('./VF_data/berlin_data.parquet')

,p_id,loc_id_end,activity_start_min,activity_end_min
0,0,69300,601,651
1,0,100000000,662,897
2,0,300992,915,916
3,0,100000000,937,1224
4,1,231788,850,1215
...,...,...,...,...
13578532,3604316,102024435,1503,1505
13578533,3604317,320942,1230,1231
13578534,3604317,102024435,1253,1293
13578535,3604318,317784,1100,1105


In [15]:
pd.read_parquet('./VF_data/rns_data_2.parquet').info()

<class 'pandas.core.frame.DataFrame'>
Index: 13477747 entries, 0 to 13578536
Data columns (total 7 columns):
 #   Column                 Dtype   
---  ------                 -----   
 0   p_id                   uint32  
 1   activity_start_min     int16   
 2   activity_duration_min  int64   
 3   activity               int64   
 4   loc_id_end             int64   
 5   activity_name_mct      category
 6   activity_end_min       int16   
dtypes: category(1), int16(2), int64(3), uint32(1)
memory usage: 527.0 MB


In [7]:
# complete rns data
df = pd.read_csv('./VF_data/region_niedersachsen_tapas_modell.csv')

In [8]:
df

,p_id,hh_id,scheme_id,score_combined,score_finance,score_time,taz_id_start,taz_has_toll_start,block_id_start,loc_id_start,...,is_home,activity_start_min,activity_duration_min,car_index,is_restricted,p_group,taz_bbr_type_start,bbr_type_home,loc_selection_motive,loc_selection_motive_supply
0,102359869,101232561,62394,0.078162,1.000000e+00,1.000000,2550,f,-1,130378,...,t,950,495,-1,f,2,73,73,0,0
1,101703905,100912769,46321,0.889043,2.403246e-16,0.999094,1896,f,-1,-100912769,...,f,415,1,0,f,17,73,73,0,0
2,101703905,100912769,46321,0.889043,2.403246e-16,0.999094,1896,f,-1,220009,...,t,418,485,0,f,17,73,73,0,0
3,101703905,100912769,46321,0.889043,2.403246e-16,0.999094,1896,f,-1,-100912769,...,f,905,185,0,f,17,73,73,0,0
4,101703905,100912769,46321,0.889043,2.403246e-16,0.999094,2019,f,-1,220132,...,f,1205,1,0,f,17,73,73,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9012231,101528174,100824986,121136,0.000031,0.000000e+00,0.000000,1686,f,-1,65468,...,t,1174,240,0,f,15,71,71,0,0
9012232,102359869,101232561,62394,0.078162,1.000000e+00,1.000000,2551,f,-1,-101232561,...,f,739,110,0,f,2,73,73,0,0
9012233,102359869,101232561,62394,0.078162,1.000000e+00,1.000000,2597,f,-1,28,...,f,851,10,0,f,2,72,73,0,0
9012234,102359869,101232561,62394,0.078162,1.000000e+00,1.000000,2639,f,-1,220752,...,t,865,30,0,f,2,72,73,0,0
